In [205]:
#初始化路径，每种方案共26个参数代表了26条edge，前8条edge为8架flight的路径
#若pairing中含有该路径则赋值为1，不含则为0
initial_cut_pattern=np.zeros((26,8))
initial_cut_pattern1 = np.array([[1,0,0,0,0,1,0,0],
                                [1,1,0,0,0,0,0,0],
                                [0,1,1,0,1,0,0,1],
                                [0,0,0,0,0,0,1,0],
                                [0,0,0,1,0,0,0,1],
                                [0,0,1,0,0,0,0,0],
                                [0,0,0,0,1,1,1,0],
                                [0,0,0,1,0,0,0,0]])
initial_cut_pattern[:8,:]=initial_cut_pattern1
#后面关于初始节点、结束节点、中介节点的边变量初始化其实可以随便写，因为只有八个dual variable，不影响subproblem求解。
#这里为了直观还是写出了结果
#1-2
initial_cut_pattern[8,0]=1
initial_cut_pattern[16,0]=1
#2-3
initial_cut_pattern[9,1]=1
initial_cut_pattern[23,1]=1
initial_cut_pattern[17,1]=1
#3-6
initial_cut_pattern[10,2]=1
initial_cut_pattern[25,2]=1
initial_cut_pattern[20,2]=1
#8-5
initial_cut_pattern[14,3]=1
initial_cut_pattern[25,3]=1
initial_cut_pattern[19,3]=1
#7-3
initial_cut_pattern[13,4]=1
initial_cut_pattern[17,4]=1
#1-7
initial_cut_pattern[8,5]=1
initial_cut_pattern[22,5]=1
initial_cut_pattern[21,5]=1
#7-4
initial_cut_pattern[13,6]=1
initial_cut_pattern[24,6]=1
initial_cut_pattern[18,6]=1
#3-5
initial_cut_pattern[10,7]=1
initial_cut_pattern[25,7]=1
initial_cut_pattern[19,7]=1

In [206]:
import gurobipy as grb
from gurobipy import GRB
import numpy as np

#定义主问题，其中x代表是否采用该pairing，1代表采用，0代表不采用
#添加约束条件，每个飞机必须至少被1个paring覆盖
#添加目标函数，最小化总cost
#返回dual variable的值，便于后续子问题求解
def master_problem(column, vtype):
    m = grb.Model()
    x = m.addMVar(shape=column.shape[1], lb=0, vtype=vtype)
    
    m.addConstr(column[:8,:] @ x >= demand_number_array) 
    m.setObjective(cost@x, GRB.MINIMIZE)
    m.optimize()

    if vtype == GRB.CONTINUOUS:
        return np.array(m.getAttr('Pi', m.getConstrs()))
    else:
        return m.objVal, np.array(m.getAttr('X'))


def restricted_lp_master_problem(column):
    return master_problem(column, GRB.CONTINUOUS)


def restricted_ip_master_problem(column):
    return master_problem(column, GRB.INTEGER)



In [207]:
#定义变量x，x表示可连接的edge，共26条。使用字典的形式，第1、2、3、4个key中的维度分别代表（出发地点，到达地点，出发时间，到达时间）
#对于地点，使用0，1，2，3，4，5分别代表起始node，DFW,LGA,ORD,RDU，结束node
#对于node时间为飞机起飞降落时间，初始点设为0，结束点设为24
def knapsack_subproblem(kk):
    global costroute,cost
    m = grb.Model()
    x = {}
    
    #前八条定义飞机飞行路线的edge
    for i in range(8):
        x[dair[i],aair[i],dep[i],arrive[i]]=m.addVar(lb=0, vtype=GRB.CONTINUOUS,name='air%s'%i)
    #定义初始节点到飞机出发节点的edge，只有出发节点可以和初始结点连接，结束结点不连接
    for i in range(8):
        if (0,dair[i],0,dep[i]) in x.keys():
            pass
        else:
            x[0,dair[i],0,dep[i]]=m.addVar(lb=0, vtype=GRB.CONTINUOUS)
    #定义飞机到达结点到结束节点的edge
    for i in range(8):
        if (aair[i],5,arrive[i],24) in x.keys():
            pass
        else:
            x[aair[i],5,arrive[i],24]=m.addVar(lb=0, vtype=GRB.CONTINUOUS)

    #定义中介edge，这部分为等待时间，不发生飞行事件
    x[2,3,13,14]=m.addVar(lb=0, vtype=GRB.CONTINUOUS)
    x[3,3,15,16]=m.addVar(lb=0, vtype=GRB.CONTINUOUS)
    x[3,3,16,17]=m.addVar(lb=0, vtype=GRB.CONTINUOUS)
    x[4,4,18,19]=m.addVar(lb=0, vtype=GRB.CONTINUOUS)


    #对每个结点添加流入=流出的约束
    #其中，限制条件为流入时间和地点都要相同的variable加和，流出时间和地点都相同的variable加和。
    for j in n:
        for des in range(4):
            lhs=LinExpr(0) 
            rhs=LinExpr(0) 
            for key in x:
                if key[3]==j and key[1]==des+1:
                    lhs.addTerms(1,x[key])
                if key[2]==j and key[0]==des+1:
                    rhs.addTerms(1,x[key])
            m.addConstr(lhs==rhs)

    #添加初始流入为1，结束流出为1的约束
    #添加飞机数》=2的约束
    #计算每条路径的cost,存放在线性表达式costroute中。对每条被选择的边的到达时间-出发时间加和，不包括包含初始和结束结点的边
    #使用flight计算每条路径包含的飞机数，对包含出发结点、结束结点的边和中介边不考虑
    start=LinExpr(0) 
    fini=LinExpr(0) 
    costroute=LinExpr(0) 
    flight=LinExpr(0) 
    for key in x:
        if key[0]!=key[1] and key[0]!=0 and key[1]!=5:
            flight.addTerms(1,x[key])
        if key[2]==0:
            start.addTerms(1,x[key])
        elif key[3]==24:
            fini.addTerms(1,x[key])
        else:
            costroute.addTerms(key[3]-key[2],x[key])
        
    m.addConstr(start==1)
    m.addConstr(fini==1)
    m.addConstr(flight>=2)
    
    #最小化cj-(cbt)*(b-1)*Pj,得到最优值
    m.setObjective(costroute+5-sum(kk[t]*(list(x.values())[t]) for t in range(8)) ,GRB.MINIMIZE)
    m.optimize()
    obj=m.ObjVal
                   
    #如果subproblem目标函数值obj<0，证明原问题未找到最优解
    flag_new_column = obj < 0
    #此时生成新的一列路径方案添加到原方案后，并且将该路径的cost放入costlist
    if flag_new_column:
        new_column = m.getAttr('X')
        cost=np.append(cost,[costroute.getValue()+5])
    #若最优，则生成新列
    else:
        new_column = None
    return flag_new_column, new_column

In [208]:
#定义初始输入参数
#出发和到达时间参数
dep=np.array([9,13,16,17,19,19,14,16])
arrive=np.array([13,15,18,19,21,21,16,18])
#最后每个飞机至少有一个paring 覆盖
demand_number_array = np.ones(8)
#初始八个方案的cost
cost=np.array([11,10,10,10,9,12,10,10])


flag_new_cut_pattern = True
new_cut_pattern = None
cut_pattern = initial_cut_pattern
#当flag_new_cut_pattern为true，即未达到最优时，添加新列进入原方案
#解master problem
#解subproblem
#判断是否最优，非最优迭代，若最优则停止
while flag_new_cut_pattern:
    if new_cut_pattern:
        cut_pattern = np.column_stack((cut_pattern, new_cut_pattern))
    kk = restricted_lp_master_problem(cut_pattern)
    flag_new_cut_pattern, new_cut_pattern = knapsack_subproblem(kk)
    
#将master problem用整数规划求解，解得的x代表是否使用该条路径，若使用则为1，不使用则为0
#对每个方案的cost加和得到最小总cost
minimal_cost, optimal_number = restricted_ip_master_problem(cut_pattern)


Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 8 rows, 8 columns and 16 nonzeros
Model fingerprint: 0x771717ec
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [9e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 8 rows and 8 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.1000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds
Optimal objective  4.100000000e+01
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 39 rows, 26 columns and 61 nonzeros
Model fingerprint: 0xb36bd69f
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 32 rows and 10 columns
Presolve time: 0.02s
Presolved: 7 rows, 16 columns, 41

Presolve time: 0.02s
Presolved: 8 rows, 14 columns, 34 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   8.000000e+00   0.000000e+00      0s
      10    3.7000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 10 iterations and 0.04 seconds
Optimal objective  3.700000000e+01
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 39 rows, 26 columns and 61 nonzeros
Model fingerprint: 0x9cb51cf2
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 32 rows and 10 columns
Presolve time: 0.02s
Presolved: 7 rows, 16 columns, 41 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -3.5000000e+30   3.000000e+30   3.500000e+00      0s
       6    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.04 seconds
Optimal objective  0.0000

In [209]:
print('************************************************')
#打印方案结果
print('result:')
print(f'minimal_cost: {minimal_cost}')
for op in range(len(optimal_number)):
    if optimal_number[op]!=0:
        print(f'filght_contained: {cut_pattern[:8,op]}')
        print(f'route_cost: {cost[op]}')
#最优方案共含有3个pairing
#① 包含飞机8-5，路径为DFW-RDU-LGA，cost为10
#② 包含飞机1-2-4，路径为DFW-LGA-DFW，cost为15
#③ 包含飞机7-3-6，路径为LGA-ORD-RDU-DFW，cost为12
#总cost为37


************************************************
result:
minimal_cost: 37.0
filght_contained: [0. 0. 0. 0. 0. 1. 0. 1.]
route_cost: 10.0
filght_contained: [1. 1. 0. 1. 0. 0. 0. 0.]
route_cost: 15.0
filght_contained: [0. 0. 1. 0. 1. 0. 1. 0.]
route_cost: 12.0
